In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers.legacy import Adam
from sklearn.metrics import accuracy_score, classification_report

In [2]:
Data = pd.read_csv('../Data/Galaxy_Data.csv')

<h1> Preprocessing <h1>

In [3]:
Data = Data[(Data['oii_3726_flux'] != 0) & (Data['neiii_3869_flux'] != 0) & (Data['oiii_5007_flux'] != 0) & (Data['h_beta_flux'] != 0) & (Data['nii_6584_flux'] != 0) & (Data['h_alpha_flux'] != 0) & (Data['sii_6731_flux'] != 0)]
Flux_Measurements = Data.drop(columns = ['Redshift', 'bptclass'])
Galaxy_Classifications = Data['bptclass']

In [4]:
def change_values(value):
    if value == 1 or value == 2:
        return 0
    if value == 3:
        return 1
    elif value == 4 or value == 5:
        return 2

In [5]:
Galaxy_Classifications = Galaxy_Classifications.apply(change_values)

In [6]:
Aggregate_Data = {'neiii_3869_flux/oii_3726_flux':Flux_Measurements['neiii_3869_flux']/Flux_Measurements['oii_3726_flux'], 
            'oiii_5007_flux/h_beta_flux' : Flux_Measurements['oiii_5007_flux']/Flux_Measurements['h_beta_flux'],  
            'nii_6584_flux/h_alpha_flux': Flux_Measurements['nii_6584_flux']/Flux_Measurements['h_alpha_flux'],
            'sii_6731_flux/h_alpha_flux':Flux_Measurements['sii_6731_flux']/Flux_Measurements['h_alpha_flux']}
Emission_Ratios = pd.DataFrame(data = Aggregate_Data)
Emission_Ratios.head()

,neiii_3869_flux/oii_3726_flux,oiii_5007_flux/h_beta_flux,nii_6584_flux/h_alpha_flux,sii_6731_flux/h_alpha_flux
0,-0.066153,0.731779,0.445568,0.223006
2,0.063194,0.836349,0.519029,0.217885
3,-0.061819,0.250297,0.298235,0.105591
4,-0.041741,0.925244,0.202580,0.202057
5,1.217840,0.294629,0.100871,-0.012745


In [7]:
X_train, X_test, y_train, y_test = train_test_split(Emission_Ratios, Galaxy_Classifications, test_size=0.2, random_state=42)

In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(Emission_Ratios.shape[1],)),  # Input layer based on the number of features
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')  # Output layer with 3 neurons for the three classes and softmax activation
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [9]:
model.fit(X_train, y_train, epochs=100, verbose=0)

In [10]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy * 100:.2f}%")

2/2 [==============================] - 0s 5ms/step - loss: 1.4020 - accuracy: 0.8108
Test Loss: 1.4020, Test Accuracy: 81.08%


In [11]:
predictions = model.predict(X_test)

2/2 [==============================] - 0s 3ms/step
